# 🌦️ Traitement des données météo avec Meteostat et Google Sheets

## 1.  Importation des bibliothèques nécessaires

In [ ]:
from datetime import datetime
import os
import json
import pandas as pd
from meteostat import Point, Daily
from google.oauth2 import service_account
from googleapiclient.discovery import build

## 2.  Initialisation des chemins et coordonnées

In [ ]:
# Coordonnées de Cape Town
cape_town = Point(-33.9249, 18.4241)

# Chemins personnalisés - à adapter selon votre environnement
base_path = os.getcwd()
data_brut_path = os.path.join(base_path, 'data/data_brut/cape_town-20-25')
data_propre_path = os.path.join(base_path, 'data/data_propre/cape_town-20-25')
data_final_csv = os.path.join(base_path, 'data/data_pret/cape_town.csv')

## 3.  Récupération des données météo de Meteostat

In [ ]:
def fetch_and_save_history(start_date: str = "2020-01-01", output_dir: str = data_brut_path):
    try:
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.now()
        os.makedirs(output_dir, exist_ok=True)
        data = Daily(cape_town, start, end).fetch()
        if data.empty:
            print(" Aucune donnée récupérée.")
            return

        for date, row in data.iterrows():
            filename = f"cape_town_{date.strftime('%Y-%m-%d')}.json"
            path = os.path.join(output_dir, filename)
            row_cleaned = row.where(pd.notnull(row), None)
            weather_info = {
                "city": "Cape Town",
                "date": date.strftime("%Y-%m-%d"),
                "temperature_avg": row_cleaned.get("tavg"),
                "temperature_min": row_cleaned.get("tmin"),
                "temperature_max": row_cleaned.get("tmax"),
                "precipitation": row_cleaned.get("prcp"),
                "snow": row_cleaned.get("snow"),
                "wind_speed": row_cleaned.get("wspd"),
                "humidity": row_cleaned.get("rhum"),
            }
            with open(path, "w") as f:
                json.dump(weather_info, f, indent=2)
            print(f" Données sauvegardées dans {path}")
    except Exception as e:
        print(f" Erreur : {e}")

## 4. Nettoyage des fichiers JSON bruts

In [ ]:
def clean_and_write(source_folder=data_brut_path, destination_folder=data_propre_path):
    os.makedirs(destination_folder, exist_ok=True)
    for filename in os.listdir(source_folder):
        if filename.endswith('.json'):
            source_path = os.path.join(source_folder, filename)
            destination_path = os.path.join(destination_folder, filename)
            with open(source_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            cleaned_data = {
                key: (0.0 if value is None else value)
                for key, value in data.items()
            }
            with open(destination_path, 'w', encoding='utf-8') as f:
                json.dump(cleaned_data, f, ensure_ascii=False, indent=2)
    print(" Nettoyage terminé.")

## 5. Fusion des fichiers JSON nettoyés en un CSV

In [ ]:
def merge_all_json_to_csv(source_folder=data_propre_path, output_path=data_final_csv):
    records = []
    for filename in os.listdir(source_folder):
        if filename.endswith('.json'):
            file_path = os.path.join(source_folder, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                if 'date' not in data:
                    continue
                data['date'] = pd.to_datetime(data['date'])
                for key in ['temperature_avg', 'temperature_min', 'temperature_max', 'precipitation', 'snow', 'wind_speed', 'humidity']:
                    data[key] = data.get(key, 0.0) or 0.0
                records.append(data)
            except Exception as e:
                print(f"Erreur dans {filename} : {e}")
    if not records:
        print("Aucune donnée trouvée.")
        return
    df = pd.DataFrame(records)
    df['date'] = df['date'].dt.date
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f" Fichier fusionné enregistré ici : {output_path}")

## 6. Upload du CSV vers Google Sheets via Drive API

In [ ]:
def get_or_create_sheet(service, sheet_name, folder_id):
    query = f"'{folder_id}' in parents and name = '{sheet_name}' and mimeType = 'application/vnd.google-apps.spreadsheet' and trashed = false"
    results = service.files().list(q=query, fields="files(id)").execute()
    files = results.get("files", [])
    if files:
        print(f" Feuille trouvée : {files[0]['id']}")
        return files[0]['id']
    else:
        file_metadata = {
            "name": sheet_name,
            "mimeType": "application/vnd.google-apps.spreadsheet",
            "parents": [folder_id]
        }
        file = service.files().create(body=file_metadata, fields="id").execute()
        print(f" Nouvelle feuille créée : {file['id']}")
        return file['id']

def write_df_to_sheet(service, spreadsheet_id, df, sheet_name='Sheet1'):
    sheets_service = service.spreadsheets()
    sheets_service.values().clear(spreadsheetId=spreadsheet_id, range=sheet_name).execute()
    values = [df.columns.tolist()] + df.values.tolist()
    sheets_service.values().update(
        spreadsheetId=spreadsheet_id,
        range=sheet_name,
        valueInputOption="RAW",
        body={"values": values}
    ).execute()
    print(f"{len(df)} lignes écrites.")

def upload_csv_to_drive(service_account_path, file_path, file_name, folder_id):
    creds = service_account.Credentials.from_service_account_file(
        service_account_path,
        scopes=['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
    )
    drive_service = build('drive', 'v3', credentials=creds)
    sheets_service = build('sheets', 'v4', credentials=creds)
    df = pd.read_csv(file_path)
    sheet_name = os.path.splitext(file_name)[0]
    spreadsheet_id = get_or_create_sheet(drive_service, sheet_name, folder_id)
    write_df_to_sheet(sheets_service, spreadsheet_id, df)

## 7.  Exécution manuelle étape par étape

In [ ]:
# fetch_and_save_history('2020-01-01')
# clean_and_write()
# merge_all_json_to_csv()
# upload_csv_to_drive('service_account.json', data_final_csv, 'cape_town.csv', 'ID_DOSSIER_GOOGLE_DRIVE')